# Argument Mining API query testing

Load the target datafile, and see how the query results work

In [2]:
# reload local package definitions for each cell
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
from dotenv import find_dotenv, load_dotenv
import requests
import pandas as pd

from arg_mine import DATA_DIR
from arg_mine.data.loaders import get_gdelt_df

In [16]:
# load the env variables to get the API key
load_dotenv(find_dotenv())
am_user_id = os.getenv("ARGUMENTEXT_USERID")
am_user_key = os.getenv("ARGUMENTEXT_KEY")

In [9]:
csv_datapath = os.path.join(DATA_DIR, "raw", "2020-climate-change-narrative")
csv_filepath = os.path.join(csv_datapath, "WebNewsEnglishSnippets.2020.csv")

url_df = get_gdelt_df(csv_filepath)
url_df.head()

INFO:2020-06-15 19:16:35,655:arg_mine.data.loaders: reading data from: /opt/workspace/data/raw/2020-climate-change-narrative/WebNewsEnglishSnippets.2020.csv


,datetime,title,headline_image_url,content_url,snippit,timestamp
0,20200115111500,Liberal MPs back Science Minister Karen Andrew...,https://static.ffx.io/images/$zoom_0.2627%2C$m...,https://www.smh.com.au/politics/federal/libera...,the science in her interview with The Age and...,2020-01-15 11:15:00
1,20200117184500,Several crowd-pullers on day two of KLF,https://www.thehindu.com/news/cities/kozhikode...,https://www.thehindu.com/news/cities/kozhikode...,"Guha, who talked about patriotism and jingois...",2020-01-17 18:45:00
2,20200106233000,Seven arrested after gas rig protest,https://www.stourbridgenews.co.uk/resources/im...,https://www.stourbridgenews.co.uk/news/nationa...,three demands for the Scottish and UK Governm...,2020-01-06 23:30:00
3,20200101111500,Australia sending aid to wildfire towns as dea...,https://bloximages.newyork1.vip.townnews.com/h...,https://www.heraldmailmedia.com/news/nation/au...,this season the worst on record and reignited ...,2020-01-01 11:15:00
4,20200107101500,"A hot, dry country caught between fire and a c...",NaN,http://global.chinadaily.com.cn/a/202001/07/WS...,", which is burned to generate electricity, wit...",2020-01-07 10:15:00


## Build the `classify` request

In [17]:
target_url = url_df.iloc[0].content_url

In [18]:
CLASSIFY_BASE_URL = "https://api.argumentsearch.com/en/classify"

timeout = 5
headers = {}
payload = {
    "topic": "Climate Change",
    "userID": am_user_id,
    "apiKey": am_user_key,
    "model": "default",
    "sentences": target_url,
    "predictStance": "true",
    "computeAttention": "true", # does not work for BERT based models
    "topicRelevance": "word2vec",  # optional
}

response = requests.post(
    CLASSIFY_BASE_URL,
    json=payload,
    headers=headers,
    timeout=timeout,
)
response.raise_for_status()

DEBUG:2020-06-15 19:27:40,749:urllib3.connectionpool: Starting new HTTPS connection (1): api.argumentsearch.com:443
DEBUG:2020-06-15 19:27:41,656:urllib3.connectionpool: https://api.argumentsearch.com:443 "POST /en/classify HTTP/1.1" 400 85


HTTPError: 400 Client Error: BAD REQUEST for url: https://api.argumentsearch.com/en/classify